## Optimization model 

In [1]:
from model import shortestPathModel

In [2]:
# model for shortest path
grid = (5,5)
sp_model = shortestPathModel(grid)

Academic license - for non-commercial use only - expires 2021-04-13
Using license file C:\Users\Apocrypse\gurobi.lic


In [3]:
# solve
sp_model.setObj([1 for i in range(40)])
sol, obj = sp_model.solve()
print('Obj: {}'.format(obj))
for i, e in enumerate(sp_model.arcs):
    if sol[i] > 1e-3:
        print(e)

Obj: 8.0
(0, 1)
(1, 2)
(2, 3)
(3, 4)
(4, 9)
(9, 14)
(14, 19)
(19, 24)


## Data Loader

In [4]:
from data import shortestpath, dataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

In [5]:
# generate data for grid network (features and costs)
n = 1000
p = 5
x, c = shortestpath.genData(n, p, grid)

In [6]:
# data split
x_train, x_test, c_train, c_test = train_test_split(x, c, test_size=0.2, random_state=246)

In [7]:
# get training data set
sp_dataset_train = dataset.optDataset(sp_model, x_train, c_train)

Optimizing...


100%|███████████████████████████████████████████████████████████████████████| 800/800 [00:01<00:00, 402.90it/s]


In [8]:
# get training data set
sp_dataset_test = dataset.optDataset(sp_model, x_test, c_test)

Optimizing...


100%|███████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 423.65it/s]


In [9]:
# get data loader
sp_loader_train = DataLoader(sp_dataset_train, batch_size=32, shuffle=True)
sp_loader_test = DataLoader(sp_dataset_test, batch_size=32, shuffle=False)

In [10]:
for i, data in enumerate(sp_loader_train):
    x, c, w, z = data
    break

In [11]:
x.shape

torch.Size([32, 5])

In [12]:
c.shape

torch.Size([32, 40])

In [13]:
w.shape

torch.Size([32, 40])

In [14]:
z.shape

torch.Size([32, 1])

## Loss Function

In [15]:
from loss import SPOPlusLoss

In [16]:
# init SPO+ loss
criterion = SPOPlusLoss()

## Linear Regression 

In [17]:
import torch
from torch import nn

In [18]:
# build linear model
class LinearRegression(nn.Module):
    
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(p, (grid[0] - 1) * grid[1] + (grid[1] - 1) * grid[0])
    
    def forward(self, x):
        out = self.linear(x)
        return out

In [19]:
# init model
lr = LinearRegression()
# cuda
if torch.cuda.is_available():
    lr = lr.cuda()

## Training 

In [20]:
# set optimizer
optimizer = torch.optim.SGD(lr.parameters(), lr=1e-2)

In [21]:
# train model
num_epochs = 200
for epoch in range(num_epochs):
    # load data
    for i, data in enumerate(sp_loader_train):
        x, c, w, z = data
        # cuda
        x, c, w, z = x.cuda(), c.cuda(), w.cuda(), z.cuda()
        # forward pass
        cp = lr(x)
        loss = criterion.apply(sp_model, cp, c, w, z).mean()
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 1 == 0:
        print('epoch {}, loss:{:.6f}'.format(epoch, loss.item()))

epoch 0, loss:7.913526
epoch 1, loss:10.170893
epoch 2, loss:9.177206
epoch 3, loss:7.889743
epoch 4, loss:9.879528
epoch 5, loss:7.333094
epoch 6, loss:7.931744
epoch 7, loss:8.710623
epoch 8, loss:7.919879
epoch 9, loss:8.295485
epoch 10, loss:6.961536
epoch 11, loss:6.963871
epoch 12, loss:7.419167
epoch 13, loss:7.247763
epoch 14, loss:7.287656
epoch 15, loss:6.196450
epoch 16, loss:6.447455
epoch 17, loss:5.998956
epoch 18, loss:6.426476
epoch 19, loss:5.210809
epoch 20, loss:5.696073
epoch 21, loss:5.181269
epoch 22, loss:5.261272
epoch 23, loss:5.443936
epoch 24, loss:5.976016
epoch 25, loss:5.324244
epoch 26, loss:5.135509
epoch 27, loss:4.404396
epoch 28, loss:4.832863
epoch 29, loss:4.791393
epoch 30, loss:4.749573
epoch 31, loss:4.612350
epoch 32, loss:5.344852
epoch 33, loss:4.019475
epoch 34, loss:4.444191
epoch 35, loss:4.220982
epoch 36, loss:3.813695
epoch 37, loss:4.022830
epoch 38, loss:3.652021
epoch 39, loss:3.510758
epoch 40, loss:2.849797
epoch 41, loss:3.650493
e